In [ ]:
import gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [ ]:
from gym.wrappers import Monitor

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import math
import copy

In [ ]:
class CartPoleAI(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc = nn.Sequential(
                        nn.Linear(4,128, bias=True),
                        nn.ReLU(),
                        nn.Linear(128,2, bias=True),
                        nn.Softmax(dim=1)
                        )

                
        def forward(self, inputs):
            x = self.fc(inputs)
            return x

In [ ]:
def init_weights(m):
    
        # nn.Conv2d weights are of shape [16, 1, 3, 3] i.e. # number of filters, 1, stride, stride
        # nn.Conv2d bias is of shape [16] i.e. # number of filters
        
        # nn.Linear weights are of shape [32, 24336] i.e. # number of input features, number of output features
        # nn.Linear bias is of shape [32] i.e. # number of output features
        
        if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.00)
                

In [ ]:
def return_random_agents(num_agents):
    
    agents = []
    for _ in range(num_agents):
        
        agent = CartPoleAI()
        
        for param in agent.parameters():
            param.requires_grad = False
            
        init_weights(agent)
        agents.append(agent)
        
        
    return agents
    

In [ ]:
def run_agents(agents):
    
    reward_agents = []
    env = gym.make("CartPole-v0")
    
    for agent in agents:
        agent.eval()
    
        observation = env.reset()
        
        r=0
        s=0
        
        for _ in range(250):
            
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env.step(action)
            r=r+reward
            
            s=s+1
            observation = new_observation

            if(done):
                break

        reward_agents.append(r)        
        #reward_agents.append(s)
        
    
    return reward_agents

In [ ]:
def return_average_score(agent, runs):
    score = 0.
    for i in range(runs):
        score += run_agents([agent])[0]
    return score/runs

In [ ]:
def run_agents_n_times(agents, runs):
    avg_score = []
    for agent in agents:
        avg_score.append(return_average_score(agent,runs))
    return avg_score

In [ ]:
def mutate(agent):

    child_agent = copy.deepcopy(agent)
    
    mutation_power = 0.02 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf
            
    for param in child_agent.parameters():
    
        if(len(param.shape)==4): #weights of Conv2D

            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    for i2 in range(param.shape[2]):
                        for i3 in range(param.shape[3]):
                            
                            param[i0][i1][i2][i3]+= mutation_power * np.random.randn()
                                
                                    

        elif(len(param.shape)==2): #weights of linear layer
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    
                    param[i0][i1]+= mutation_power * np.random.randn()
                        

        elif(len(param.shape)==1): #biases of linear layer or conv layer
            for i0 in range(param.shape[0]):
                
                param[i0]+=mutation_power * np.random.randn()

    return child_agent

In [ ]:
def return_children(agents, sorted_parent_indexes, elite_index):
    
    children_agents = []
    
    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)-1):
        
        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index]))

    #now add one elite
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one
    
    return children_agents, elite_index

In [ ]:
def add_elite(agents, sorted_parent_indexes, elite_index=None, only_consider_top_n=10):
    
    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]
    
    if(elite_index is not None):
        candidate_elite_index = np.append(candidate_elite_index,[elite_index])
        
    top_score = None
    top_elite_index = None
    
    for i in candidate_elite_index:
        score = return_average_score(agents[i],runs=5)
        print("Score for elite i ", i, " is ", score)
        
        if(top_score is None):
            top_score = score
            top_elite_index = i
        elif(score > top_score):
            top_score = score
            top_elite_index = i
            
    print("Elite selected with index ",top_elite_index, " and score", top_score)
    
    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent
    

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
game_actions = 2 #2 actions possible: left or right

#disable gradients as we will not use them
torch.set_grad_enabled(False)

# initialize N number of agents
num_agents = 500
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 20

# run evolution until X generations
generations = 100

elite_index = None

for generation in range(generations):

    # return rewards of agents
    rewards = run_agents_n_times(agents, 3) #return average of 3 runs

    # sort by rewards
    sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit] #reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order
    print("")
    print("")
    
    top_rewards = []
    for best_parent in sorted_parent_indexes:
        top_rewards.append(rewards[best_parent])
    
    print("Generation ", generation, " | Mean rewards: ", np.mean(rewards), " | Mean of top 5: ",np.mean(top_rewards[:5]))
    #print(rewards)
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",top_rewards)
    
    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    # kill all agents, and replace them with their children
    agents = children_agents



Generation  0  | Mean rewards:  22.166  | Mean of top 5:  50.8
Top  20  scores [370 174 490 447 352 213 387  14 390 239 337 458 235 230 415 344 120 186
   5 327]
Rewards for top:  [53.666666666666664, 53.333333333333336, 49.333333333333336, 49.0, 48.666666666666664, 47.333333333333336, 45.666666666666664, 45.0, 44.0, 43.666666666666664, 42.0, 41.666666666666664, 41.333333333333336, 41.0, 40.0, 39.666666666666664, 39.333333333333336, 39.333333333333336, 39.333333333333336, 39.0]
Score for elite i  370  is  21.2
Score for elite i  174  is  29.2
Score for elite i  490  is  17.6
Score for elite i  447  is  15.2
Score for elite i  352  is  21.4
Score for elite i  213  is  35.6
Score for elite i  387  is  19.6
Score for elite i  14  is  18.8
Score for elite i  390  is  15.0
Score for elite i  239  is  26.4
Elite selected with index  213  and score 35.6


Generation  1  | Mean rewards:  23.85333333333333  | Mean of top 5:  51.66666666666667
Top  20  scores [446 243 205 496 362 258 135 400 4

Top  20  scores [487 338 110 235 424 122  84 366 291  36  33 287  99  68 221 409  35 228
 486 286]
Rewards for top:  [97.0, 83.0, 82.66666666666667, 78.66666666666667, 75.0, 72.0, 69.66666666666667, 67.66666666666667, 66.0, 65.66666666666667, 65.33333333333333, 65.33333333333333, 62.0, 62.0, 60.666666666666664, 60.666666666666664, 59.333333333333336, 58.666666666666664, 58.0, 57.333333333333336]
Score for elite i  487  is  40.2
Score for elite i  338  is  40.8
Score for elite i  110  is  22.2
Score for elite i  235  is  22.8
Score for elite i  424  is  40.6
Score for elite i  122  is  58.8
Score for elite i  84  is  33.6
Score for elite i  366  is  25.4
Score for elite i  291  is  43.6
Score for elite i  36  is  31.2
Score for elite i  499  is  44.6
Elite selected with index  122  and score 58.8


Generation  10  | Mean rewards:  35.502  | Mean of top 5:  76.26666666666667
Top  20  scores [401 169 356 185  80 231 123 454 203 121 243 317 455 139 318 297 494 225
 116  78]
Rewards for top

 310 276]
Rewards for top:  [121.33333333333333, 113.33333333333333, 112.0, 110.33333333333333, 109.66666666666667, 109.33333333333333, 103.33333333333333, 102.33333333333333, 100.66666666666667, 99.0, 96.66666666666667, 96.33333333333333, 96.0, 95.66666666666667, 91.66666666666667, 91.33333333333333, 90.66666666666667, 89.66666666666667, 88.66666666666667, 88.33333333333333]
Score for elite i  143  is  31.6
Score for elite i  198  is  64.6
Score for elite i  353  is  24.8
Score for elite i  466  is  38.4
Score for elite i  7  is  21.4
Score for elite i  184  is  62.0
Score for elite i  423  is  45.4
Score for elite i  117  is  91.2
Score for elite i  206  is  59.0
Score for elite i  471  is  37.6
Score for elite i  499  is  23.0
Elite selected with index  117  and score 91.2


Generation  19  | Mean rewards:  48.477333333333334  | Mean of top 5:  110.86666666666667
Top  20  scores [224 411 278 445 409 203 313  25 172 314 370 302 188 484 211 396 464  41
 365  39]
Rewards for top:  [135

Generation  27  | Mean rewards:  64.64466666666667  | Mean of top 5:  130.73333333333335
Top  20  scores [252 117 315 495 302 477  18  32 412 454  15 416 258 162 392 176 168 422
 267 209]
Rewards for top:  [144.66666666666666, 138.33333333333334, 127.0, 123.33333333333333, 120.33333333333333, 120.33333333333333, 119.0, 118.0, 115.0, 114.66666666666667, 114.0, 113.66666666666667, 113.33333333333333, 112.33333333333333, 111.66666666666667, 111.66666666666667, 110.66666666666667, 109.33333333333333, 109.0, 108.0]
Score for elite i  252  is  49.4
Score for elite i  117  is  98.8
Score for elite i  315  is  37.4
Score for elite i  495  is  70.0
Score for elite i  302  is  90.8
Score for elite i  477  is  64.8
Score for elite i  18  is  40.8
Score for elite i  32  is  64.6
Score for elite i  412  is  56.0
Score for elite i  454  is  70.0
Score for elite i  499  is  81.0
Elite selected with index  117  and score 98.8


Generation  28  | Mean rewards:  67.174  | Mean of top 5:  128.2
Top  20  

Score for elite i  499  is  111.0
Elite selected with index  37  and score 118.6


Generation  36  | Mean rewards:  88.3  | Mean of top 5:  165.73333333333332
Top  20  scores [254 462 100 431 349 178 295 162  61 200 191 241 482 415 485 494 170 312
 477  26]
Rewards for top:  [177.33333333333334, 172.33333333333334, 163.33333333333334, 158.66666666666666, 157.0, 155.66666666666666, 155.33333333333334, 150.33333333333334, 147.33333333333334, 147.33333333333334, 147.0, 145.33333333333334, 145.0, 141.66666666666666, 141.66666666666666, 140.0, 138.0, 137.66666666666666, 137.66666666666666, 137.0]
Score for elite i  254  is  95.0
Score for elite i  462  is  83.8
Score for elite i  100  is  86.4
Score for elite i  431  is  88.8
Score for elite i  349  is  94.0
Score for elite i  178  is  70.8
Score for elite i  295  is  137.2
Score for elite i  162  is  88.6
Score for elite i  61  is  101.6
Score for elite i  200  is  70.8
Score for elite i  499  is  85.6
Elite selected with index  295  and s

Score for elite i  61  is  91.4
Score for elite i  80  is  115.8
Score for elite i  365  is  94.4
Score for elite i  243  is  137.6
Score for elite i  323  is  85.0
Score for elite i  499  is  153.8
Elite selected with index  499  and score 153.8


Generation  45  | Mean rewards:  103.702  | Mean of top 5:  178.73333333333335
Top  20  scores [449 146 314 475 423  47 331 404 335 110 315 341 311 169 393 274  19 130
 324 352]
Rewards for top:  [189.66666666666666, 184.33333333333334, 179.33333333333334, 170.33333333333334, 170.0, 167.66666666666666, 167.66666666666666, 164.66666666666666, 164.33333333333334, 164.33333333333334, 162.0, 161.66666666666666, 160.66666666666666, 160.66666666666666, 160.33333333333334, 158.66666666666666, 158.66666666666666, 158.66666666666666, 157.0, 157.0]
Score for elite i  449  is  135.0
Score for elite i  146  is  126.8
Score for elite i  314  is  64.6
Score for elite i  475  is  73.6
Score for elite i  423  is  131.0
Score for elite i  47  is  87.0
Score 

Score for elite i  158  is  140.6
Score for elite i  415  is  146.0
Score for elite i  426  is  166.4
Score for elite i  462  is  115.0
Score for elite i  293  is  164.4
Score for elite i  148  is  101.6
Score for elite i  361  is  149.4
Score for elite i  423  is  142.6
Score for elite i  499  is  139.6
Elite selected with index  471  and score 178.0


Generation  54  | Mean rewards:  136.49533333333332  | Mean of top 5:  199.13333333333335
Top  20  scores [209 214 324 295 253 255 393  84  69 244 311 386  87 435 248  97 442 454
 417  25]
Rewards for top:  [200.0, 200.0, 199.33333333333334, 198.33333333333334, 198.0, 197.0, 194.33333333333334, 194.0, 193.66666666666666, 192.33333333333334, 192.0, 190.33333333333334, 189.66666666666666, 189.66666666666666, 189.66666666666666, 189.0, 188.66666666666666, 188.33333333333334, 188.33333333333334, 187.66666666666666]
Score for elite i  209  is  113.4
Score for elite i  214  is  146.6
Score for elite i  324  is  141.0
Score for elite i  295  i

KeyboardInterrupt: 

In [ ]:
def play_agent(agent):
        env = gym.make("CartPole-v0")
        
        env_record = Monitor(env, './video', force=True)
        observation = env_record.reset()
        last_observation = observation
        r=0
        for _ in range(250):
            env_record.render()
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
             =output_probabilities agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env_record.step(action)
            r=r+reward
            observation = new_observation

            if(done):
                break

        env_record.close()
        print("Rewards: ",r)      

In [ ]:
play_agent(agents[2])

Rewards:  141.0
